In [ ]:
import os
import jax

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '1.0'
jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np
import plotly.io as pio
pio.renderers.default = "browser"

import matplotlib.pyplot as plt
import plotly.graph_objects as go
# import plotly.subplots as sp

import cbvf_reachability as cbvf

from tqdm import tqdm
from dyn_sys.DvNonlinearCar import DvNonlinearCar
from controllers.CBVF_QP import CBVFQPController
from controllers.CBVFInterpolator import CBVFInterpolator
from utils.animator import create_trajectory_animation



In [ ]:
car_params = {'m': 1708, 'Vx': 50.0 / 3.6, 'Lf': 1.536, 'Lr': 1.575, 'Iz': 2985.216, 'mu': 1.0, 'Mz': 1000,
              'Cf': 1.5745 * 10e5, 'Cr': 1.6426 * 10e5, 'Wf': 0.75, 'Wr': 0.75}
dynamics = DvNonlinearCar(car_params=car_params)

# limits of the grid in degrees
x1_lim = 200
x2_lim = 45
x3_lim = 25

x1_lim = x1_lim * jnp.pi / 180
x2_lim = x2_lim * jnp.pi / 180
x3_lim = x3_lim * jnp.pi / 180

grid = cbvf.Grid.from_lattice_parameters_and_boundary_conditions(cbvf.sets.Box(np.array([-x1_lim, -x2_lim, -x3_lim]),
                                                                           np.array([x1_lim, x2_lim, x3_lim]),),
                                                                           (65, 65, 65))
values_vi = jnp.linalg.norm(grid.states[..., :3], axis=-1) - 6 * jnp.pi / 180
# values_vi = jnp.zeros(grid.shape) + 1
# values_vi = jnp.where(-jnp.linalg.norm(grid.states[..., :2], axis=-1) + 6 >= 0, values_vi*10, values_vi*0)

initial_values = jnp.linalg.norm(grid.states[..., :3], axis=-1) - 6 * (jnp.pi / 180)

# initial_values = jnp.zeros_like(values_vi) + 100

times = np.linspace(0, -0.25, 160)

gamma = 5.0

In [ ]:
jax.clear_caches()
solver_settings = cbvf.SolverSettings.with_accuracy("cbvf",
                                                  hamiltonian_postprocessor=cbvf.solver.identity,
                                                  gamma=gamma)
cbvf_values = -cbvf.solve_cbvf(solver_settings=solver_settings,
                              dynamics=dynamics,
                              grid=grid,
                              times=times,
                              initial_values=values_vi,
                              target_values=initial_values,)


# solver_settings = cbvf.SolverSettings.with_accuracy(
#     "very_high",
#     hamiltonian_postprocessor=cbvf.solver.backwards_reachable_tube,
# )
# target_values = cbvf.solve(solver_settings, dynamics, grid, times, initial_values)

In [ ]:
# for i, time in enumerate(times):
plt.jet()
plt.figure(figsize=(13, 13))
plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], cbvf_values[-15, :, :].T)
plt.colorbar()
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            cbvf_values[-15, :, :].T,
            levels=0,
            colors="black",
            linewidths=3)
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            values_vi.T,
            levels=0,
            colors="white",
            linewidths=3,
            linestyles='--')
plt.title(f"Results for time {times[-15]} s")
plt.show()



In [ ]:
f = go.Figure(data=go.Surface(z=cbvf_values[-1, :, :].T,
                              x=grid.coordinate_vectors[0],
                              y=grid.coordinate_vectors[1],
                              colorscale="Jet",
                              contours={
                                             "z": {
                                                 "show": True,
                                                 "start": 0.00,
                                                 "end": 0.00,
                                                 "size": 1
                                             }
                                         }))
f.update_layout(title=f"CBVF Nonlinear Bicycle with gamma {gamma}",
                  scene=dict(xaxis_title="x1",
                             yaxis_title="x2",
                             zaxis_title="value",
                             xaxis=dict(range=[-x1_lim, x1_lim]),
                             yaxis=dict(range=[-x2_lim, x2_lim]),
                             zaxis=dict(range=[-4, 2.0])),
                  width=800,
                  height=800,
                  scene_aspectmode='cube',
                  font=dict(size=22),)
f.show()

In [ ]:
# Downsample the data for faster plotting
downsample_factor = 4  # Use every 2nd point
cbvf_downsampled = cbvf_values[::1, ::downsample_factor, ::downsample_factor]
x_downsampled = grid.coordinate_vectors[0][::downsample_factor]
y_downsampled = grid.coordinate_vectors[1][::downsample_factor]

# Create frames (keeping your existing logic but with downsampled data)
frames = []
n_slices = cbvf_downsampled.shape[0]

for i in range(n_slices):
    if i % 5 != 0:
        continue
    frame = go.Frame(
        data=[go.Surface(z=cbvf_downsampled[i, :, :].T,
                        x=x_downsampled,
                        y=y_downsampled,
                        colorscale="Jet")],  # Removed contours for speed
        name=str(i)
    )
    frames.append(frame)

# Create initial figure
f = go.Figure(
    data=[go.Surface(z=cbvf_downsampled[0, :, :].T,
                    x=x_downsampled,
                    y=y_downsampled,
                    colorscale="Jet")],
    frames=frames
)

# Add slider
sliders = [dict(
    steps=[dict(args=[[str(i)], {'frame': {'duration': 0, 'redraw': True}}],
                label=str(i), method='animate') for i in range(n_slices)],
    active=0,
    currentvalue={"prefix": "Slice: "},
    pad={"t": 50},
)]

f.update_layout(
    title=f"CBVF Double Integrator with gamma {gamma}",
    scene=dict(xaxis_title="x1",
               yaxis_title="x2",
               zaxis_title="value",
               xaxis=dict(range=[-x1_lim, x1_lim]),
               yaxis=dict(range=[-x2_lim, x2_lim]),
               zaxis=dict(range=[-6, 3])),
    width=800,
    height=800,
    scene_aspectmode='cube',
    sliders=sliders
)

f.show()

In [ ]:
# Get device references
cpu = jax.devices('cpu')[0]

# Move data to CPU and switch computation
target_values_cpu = jax.device_put(cbvf_values, cpu)
grid_cpu = jax.device_put(grid, cpu)
times_cpu = jax.device_put(times, cpu)

# Check both gradient components at safe region entry points
gradient_x1 = np.zeros(grid_cpu.shape)
gradient_x2 = np.zeros(grid_cpu.shape)
entry_times_grid = np.zeros(grid_cpu.shape)
entry_values_grid = np.zeros(grid_cpu.shape)

with jax.default_device(cpu):

    interpolator = CBVFInterpolator(grid=grid_cpu,
                                    cbvf_values=target_values_cpu,  # Use CPU version
                                    times=times_cpu, gamma=gamma)

    states_flat = grid_cpu.states.reshape(-1, 2)

    # Compute gradients at safe region entry points
    entry_times, entry_values, spatial_grads, time_grads = interpolator.compute_safe_entry_gradients_efficient(states_flat)
    # spatial_grads = interpolator.interpolate_spatial_gradient_vectorized(states_flat, times_cpu[-2])

    # Reshape results back to grid shape
    gradient_x1 = spatial_grads[:, 0].reshape(grid.shape)
    gradient_x2 = spatial_grads[:, 1].reshape(grid.shape)
    time_grads = time_grads.reshape(grid.shape)
    entry_times_grid = entry_times.reshape(grid.shape)
    entry_values_grid = entry_values.reshape(grid.shape)


In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('∂V/∂x1', '∂V/∂x2', '∂V/∂t'),
    specs=[[{'type': 'surface'}, {'type': 'surface'}], [{'type': 'surface'}, {'type': 'surface'}]]
)

# Add ∂V/∂x1
fig.add_trace(
    go.Surface(
        z=gradient_x1,
        x=grid.coordinate_vectors[0],
        y=grid.coordinate_vectors[1],
        colorscale="RdBu",
        name="∂V/∂x1",
        contours={"z": {"show": True, "start": 0.0, "end": 0.0, "size": 1}}
    ),
    row=1, col=1
)

# Add ∂V/∂x2
fig.add_trace(
    go.Surface(
        z=gradient_x2,
        x=grid.coordinate_vectors[0],
        y=grid.coordinate_vectors[1],
        colorscale="RdBu",
        name="∂V/∂x2",
        contours={"z": {"show": True, "start": 0.0, "end": 0.0, "size": 1}}
    ),
    row=1, col=2
)

# Add ∂V/∂t
fig.add_trace(
    go.Surface(
        z=time_grads,
        x=grid.coordinate_vectors[0],
        y=grid.coordinate_vectors[1],
        colorscale="RdBu",
        name="∂V/∂t",
        contours={"z": {"show": True, "start": 0.0, "end": 0.0, "size": 1}}
    ),
    row=2, col=1
)

fig.update_layout(
    title=f"Gradients of the CBVF Value Function (gamma={gamma})",
    scene=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="∂V/∂x1"),
    scene2=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="∂V/∂x2"),
    height=1000,
    font=dict(size=18),
)
fig.show()

In [ ]:
# plot the time of entry of every point into the safe set
plt.jet()
plt.figure(figsize=(13, 13))
plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], entry_times_grid.T)
plt.colorbar()
plt.show()

In [ ]:
x_init = jnp.array([0.0, 0.2])
target = np.array([-70 * (jnp.pi/180), -30 * (jnp.pi/180)])

# Get device references
cpu = jax.devices('cpu')[0]

# Move data to CPU and switch computation
cbvf_values_cpu = jax.device_put(cbvf_values, cpu)
grid_cpu = jax.device_put(grid, cpu)
times_cpu = jax.device_put(times, cpu)


with jax.default_device(cpu):

    # coefficients of a PD controller
    k_p = 200000
    k_d = 500000

    interpolator = CBVFInterpolator(grid=grid_cpu,
                                    cbvf_values=cbvf_values_cpu,
                                    times=times_cpu, gamma=100)

    safe_controller = CBVFQPController(value_fn=interpolator,
                                       gamma=100,
                                       verbose=False,)

    del cbvf_values_cpu, grid_cpu

    sim_time = np.linspace(0, 0.5, 500)
    dt = sim_time[1] - sim_time[0]

    trajectory = np.zeros((len(sim_time), 2))
    trajectory_pd = np.zeros((len(sim_time), 2))
    control_his = np.zeros(len(sim_time))
    control_ref = np.zeros(len(sim_time))
    constrain_his = np.zeros(len(sim_time))
    cbvf_vals = np.zeros(len(sim_time))
    cbvf_grad_x = np.zeros((len(sim_time), 2))
    cbvf_grad_t = np.zeros(len(sim_time))
    a_term = np.zeros(len(sim_time))
    constrain_rest_hist = np.zeros(len(sim_time))

    state = x_init.copy()
    state_pd = x_init.copy()

    for i, time in enumerate(tqdm(sim_time, desc="Simulation", unit="sim_s", unit_scale=dt)):
        grad_time = interpolator.compute_safe_entry_gradients_efficient(jnp.array(state).reshape(1, -1))[0]
        # _, cbvf_grad_x[i], cbvf_grad_t[i] = safe_controller.cbvf.get_value_and_gradient(state, grad_time[0])
        state_flat = state.reshape(-1, 2)

        # Compute gradients at safe region entry points
        _, _, list1, list2 = interpolator.compute_safe_entry_gradients_efficient(state_flat)
        cbvf_grad_t[i] = list2[0]
        cbvf_grad_x[i, :] = list1[0]
        cbvf_vals[i], _, _ = safe_controller.cbvf.get_value_and_gradient(state, times_cpu[-1])
        # u = k_p * (target[0] - state[0]) + k_d * (target[1] - state[1])
        u = -15000
        control_ref[i] = u
        u_prev = 0.0 if i == 0 else control_his[i - 1]
        u_safe, constrain_his[i], a_term[i] = safe_controller.compute_safe_control(state=state, time=time,
                                                      u_ref=u, dynamics=dynamics,
                                                      u_prev=u_prev,
                                                      u_max_mag = 15000,
                                                      gradient_time=grad_time,)
        state_dot = dynamics.open_loop_dynamics(state=state, time=time) + dynamics.control_jacobian(state, time) @ np.array([u_safe])
        state_dot_pd = dynamics.open_loop_dynamics(state=state_pd, time=time) + dynamics.control_jacobian(state_pd, time) @ np.array([u])
        state = state + state_dot * dt
        state_pd = state_pd + state_dot_pd * dt

        trajectory[i, :] = state
        trajectory_pd[i, :] = state_pd
        control_his[i] = u_safe

    constrain_rest_hist = constrain_his - a_term



In [ ]:
plt.rcParams.update({'font.size': 16})

# Plot results
fig, axes = plt.subplots(8, 1, figsize=(16, 16))

# State trajectory
axes[0].plot(sim_time, trajectory[:, 0], label='Yaw Rate (x1)')
axes[0].plot(sim_time, trajectory[:, 1], label='Side Slip (x2)')
axes[0].set_ylabel('States')
axes[0].legend()
axes[0].grid(True)

# Control input
axes[1].plot(sim_time, control_his, 'r-', label='Control Input')
axes[1].axhline(dynamics.control_space.lo, color='black', linestyle='--', label='Control Limit')
axes[1].axhline(dynamics.control_space.hi, color='black', linestyle='--')
axes[1].plot(sim_time, control_ref, 'g--', label='Reference Control')
axes[1].set_ylabel('Control')
axes[1].legend()
axes[1].grid(True)

# QP Constraints
axes[2].plot(sim_time, constrain_his, 'r-', label='QP Constraint')
axes[2].axhline(0, color='black', linestyle='--', label='Zero Constraint')
axes[2].set_ylabel('QP Constraint')
axes[2].legend()
axes[2].grid(True)

# CBVF Values
axes[3].plot(sim_time, cbvf_vals, 'r-', label='CBVF Value')
axes[3].axhline(0, color='black', linestyle='--', label='Zero Constraint')
axes[3].set_ylabel('CBVF Value')
axes[3].legend()
axes[3].grid(True)

# CBVF Gradients X
axes[4].plot(sim_time, cbvf_grad_x[:, 0], 'r-', label='CBVF Grad X1')
axes[4].plot(sim_time, cbvf_grad_x[:, 1], 'b-', label='CBVF Grad X2')
axes[4].set_ylabel('CBVF Gradients')
axes[4].legend()
axes[4].grid(True)

# CBVF Gradients Tgamma
axes[5].plot(sim_time, cbvf_grad_t, 'r-', label='CBVF Grad T')
axes[5].set_ylabel('CBVF Gradients')
axes[5].legend()
axes[5].grid(True)

# A Term in Constraint
axes[6].plot(sim_time, a_term, 'r-', label='"Natural Evolution" Constraint')
axes[6].set_ylabel('Value')
axes[6].legend()
axes[6].grid(True)

# Controlled Constraint
axes[7].plot(sim_time, constrain_rest_hist, 'r-', label='Controlled Constraint')
axes[7].set_ylabel('Value')
axes[7].legend()
axes[7].grid(True)

plt.tight_layout()
plt.show()

# Phase portrait
plt.figure(figsize=(10, 10))
plt.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2, label='With Safety Controller')
plt.plot(trajectory_pd[:, 0], trajectory_pd[:, 1], 'k--', linewidth=2, label='No Safety Controller')
plt.scatter(x_init[0], x_init[1], color='g', s=100, marker='o', label='Start')
plt.scatter(trajectory[-1, 0], trajectory[-1, 1], color='r', s=100, marker='s', label='End')
# plt.scatter(target[0], target[1], color='y', s=100, marker='o', label='Target')
# plt.plot(trajectory_pd[:, 0], trajectory_pd[:, 1], 'k--', linewidth=2, label='PD Trajectory')


plt.xlabel('Yaw Rate (x1)')
plt.ylabel('Side Slip (x2)')
plt.title('Phase Portrait')
plt.legend()
plt.grid(True)
# plt.axis('equal')
plt.xlim(-x1_lim, x1_lim)
plt.ylim(-x2_lim, x2_lim)
# Capture the contour objects
contour1 = plt.contour(grid.coordinate_vectors[0],
                      grid.coordinate_vectors[1],
                      cbvf_values[-1, :, :].T,
                      levels=0,
                      colors="black",
                      linewidths=3)

contour2 = plt.contour(grid.coordinate_vectors[0],
                      grid.coordinate_vectors[1],
                      values_vi.T,
                      levels=0,
                      colors="green",
                      linewidths=3,
                      linestyles='--')

# Add manual legend entries for the contours
from matplotlib.lines import Line2D
legend_elements = plt.gca().get_legend_handles_labels()[0] + [
    Line2D([0], [0], color='black', linewidth=3, label='Safe Set Boundary (BRS)'),
    Line2D([0], [0], color='green', linewidth=3, linestyle='--', label='Initial Value')
]

plt.legend(handles=legend_elements)

In [ ]:
# 3D Trajectory Visualization on Value Function Surface
import plotly.graph_objects as go

# Use the already computed CBVF values along the trajectory
trajectory_z = cbvf_vals

# Convert JAX arrays to numpy for compatibility
value_function_surface = np.array(cbvf_values[-1, :, :].T)

# Create the figure with the value function surface
fig = go.Figure()

# Add the value function surface
fig.add_trace(go.Surface(
    z=value_function_surface,
    x=grid.coordinate_vectors[0],
    y=grid.coordinate_vectors[1],
    colorscale="Jet",
    opacity=0.8,
    name="Value Function",
    contours={
        "z": {
            "show": True,
            "start": 0.00,
            "end": 0.00,
            "size": 1
        }
    }
))

# Add trajectory as a 3D line using computed CBVF values
fig.add_trace(go.Scatter3d(
    x=trajectory[:, 0],  # position
    y=trajectory[:, 1],  # velocity
    z=trajectory_z,      # computed CBVF values along trajectory
    mode='lines+markers',
    line=dict(color='white', width=8),
    marker=dict(size=3, color='white'),
    name='Trajectory',
    hovertemplate='<b>Trajectory Point</b><br>' +
                  'Time: %{customdata[0]:.3f} s<br>' +
                  'Position: %{x:.3f}<br>' +
                  'Velocity: %{y:.3f}<br>' +
                  'CBVF Value: %{z:.3f}<br>' +
                  'Control Input: %{customdata[1]:.3f}<br>' +
                  '<extra></extra>',
    customdata=np.column_stack([sim_time, control_his])
))

# Add start point
fig.add_trace(go.Scatter3d(
    x=[x_init[0]],
    y=[x_init[1]],
    z=[cbvf_vals[0]],  # Use first CBVF value
    mode='markers',
    marker=dict(color='lime', size=15, symbol='circle'),
    name='Start'
))

# Add end point
fig.add_trace(go.Scatter3d(
    x=[trajectory[-1, 0]],
    y=[trajectory[-1, 1]],
    z=[cbvf_vals[-1]],  # Use last CBVF value
    mode='markers',
    marker=dict(color='red', size=15, symbol='square'),
    name='End'
))

# Add target point (use interpolator for target since it's not on trajectory)
from scipy.interpolate import RegularGridInterpolator
interpolator_target = RegularGridInterpolator(
    (grid.coordinate_vectors[1], grid.coordinate_vectors[0]),
    value_function_surface,  # Already converted to numpy
    bounds_error=False,
    fill_value=0
)
target_z = interpolator_target([target[1], target[0]])
fig.add_trace(go.Scatter3d(
    x=[target[0]],
    y=[target[1]],
    z=[target_z],
    mode='markers',
    marker=dict(color='yellow', size=15, symbol='diamond'),
    name='Target'
))

# Update layout
fig.update_layout(
    title=f"3D Trajectory on CBVF Surface (γ={gamma})",
    scene=dict(
        xaxis_title="Position (x1)",
        yaxis_title="Velocity (x2)",
        zaxis_title="Value Function",
        xaxis=dict(range=[-x1_lim, x1_lim]),
        yaxis=dict(range=[-x2_lim, x2_lim]),
        zaxis=dict(range=[0, 6]),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    width=1000,
    height=800,
    scene_aspectmode='cube'
)

fig.show()

In [ ]:
from sklearn.metrics import r2_score
from skimage import measure

cpu = jax.devices('cpu')[0]
# Move data to CPU but keep grid object structure
target_values_cpu = np.array(jax.device_put(cbvf_values, cpu))
grid_cpu = jax.device_put(grid, cpu)  # Keep as grid object, don't convert to numpy
times_cpu = np.array(jax.device_put(times, cpu))

def check_parabolic_boundaries(grid, cbvf_values, initial_circle_radius=6, plot=True):
    """
    Check if the boundaries extending beyond the initial circle are parabolic.
    Fits separate parabolas for x >= 0 and x < 0 regions.
    """
    # Convert JAX arrays to numpy arrays to avoid buffer errors
    try:
        import jax.numpy as jnp
        is_jax_available = True
    except ImportError:
        is_jax_available = False

    # Convert to numpy arrays
    if is_jax_available and hasattr(cbvf_values, '__array__'):
        cbvf_values_np = np.asarray(cbvf_values)
    else:
        cbvf_values_np = np.asarray(cbvf_values)

    # Handle grid coordinates
    try:
        x_coords_grid = np.asarray(grid.coordinate_vectors[0])
        y_coords_grid = np.asarray(grid.coordinate_vectors[1])
    except:
        x_coords_grid = np.asarray(grid.coordinate_vectors[0])
        y_coords_grid = np.asarray(grid.coordinate_vectors[1])

    # Extract zero-level contour from final value function
    final_values = cbvf_values_np[-1, :, :].T
    contours = measure.find_contours(final_values, 0.0)

    if len(contours) == 0:
        print("No zero contours found!")
        return None

    # Use the longest contour (main boundary)
    main_contour = max(contours, key=len)

    # Convert contour indices to actual coordinates
    x_coords = np.interp(main_contour[:, 1], np.arange(len(x_coords_grid)), x_coords_grid)
    y_coords = np.interp(main_contour[:, 0], np.arange(len(y_coords_grid)), y_coords_grid)

    # Filter points that are NOT on the original circle
    distances_from_origin = np.sqrt(x_coords**2 + y_coords**2)
    circle_threshold = 0.05
    non_circle_mask = np.abs(distances_from_origin - initial_circle_radius) > circle_threshold
    parabolic_x = x_coords[non_circle_mask]
    parabolic_y = y_coords[non_circle_mask]

    if len(parabolic_x) < 10:
        print("Not enough non-circular boundary points found")
        return None

    # Split points based on x-coordinate
    right_mask = parabolic_y >= 0
    left_mask = parabolic_y < 0

    right_x, right_y = parabolic_x[right_mask], parabolic_y[right_mask]
    left_x, left_y = parabolic_x[left_mask], parabolic_y[left_mask]

    def fit_horizontal_parabola(x, y):
        """Fit horizontal parabola y = ax² + bx + c"""
        if len(x) < 3:
            return None, 0
        try:
            A = np.column_stack([x**2, x, np.ones(len(x))])
            coeffs, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)
            if residuals.size > 0:
                r2 = 1 - residuals[0] / np.var(y) / len(x)
            else:
                y_pred = A @ coeffs
                r2 = r2_score(y, y_pred)
            return coeffs, r2
        except:
            return None, 0

    def fit_vertical_parabola(x, y):
        """Fit vertical parabola x = ay² + by + c"""
        if len(y) < 3:
            return None, 0
        try:
            A = np.column_stack([y**2, y, np.ones(len(y))])
            coeffs, residuals, rank, s = np.linalg.lstsq(A, x, rcond=None)
            if residuals.size > 0:
                r2 = 1 - residuals[0] / np.var(x) / len(x)
            else:
                x_pred = A @ coeffs
                r2 = r2_score(x, x_pred)
            return coeffs, r2
        except:
            return None, 0

    # Fit parabolas to each half
    right_h_coeffs, right_h_r2 = fit_horizontal_parabola(right_x, right_y)
    right_v_coeffs, right_v_r2 = fit_vertical_parabola(right_x, right_y)

    left_h_coeffs, left_h_r2 = fit_horizontal_parabola(left_x, left_y)
    left_v_coeffs, left_v_r2 = fit_vertical_parabola(left_x, left_y)

    results = {
        'num_points_total': len(parabolic_x),
        'num_points_right': len(right_x),
        'num_points_left': len(left_x),
        'right_horizontal_fit': {'coeffs': right_h_coeffs, 'r2': right_h_r2},
        'right_vertical_fit': {'coeffs': right_v_coeffs, 'r2': right_v_r2},
        'left_horizontal_fit': {'coeffs': left_h_coeffs, 'r2': left_h_r2},
        'left_vertical_fit': {'coeffs': left_v_coeffs, 'r2': left_v_r2},
        'boundary_points': (parabolic_x, parabolic_y),
        'right_points': (right_x, right_y),
        'left_points': (left_x, left_y),
        'full_contour': (x_coords, y_coords)
    }

    if plot:
        plt.figure(figsize=(15, 5))

        # Plot 1: Full boundary with original circle
        plt.subplot(1, 3, 1)
        plt.plot(x_coords, y_coords, 'b-', linewidth=2, label='Computed Boundary')
        circle_theta = np.linspace(0, 2*np.pi, 100)
        circle_x = initial_circle_radius * np.cos(circle_theta)
        circle_y = initial_circle_radius * np.sin(circle_theta)
        plt.plot(circle_x, circle_y, 'g--', linewidth=2, label=f'Original Circle (r={initial_circle_radius})')
        plt.scatter(right_x, right_y, c='red', s=10, alpha=0.7, label='Top half (y≥0)')
        plt.scatter(left_x, left_y, c='orange', s=10, alpha=0.7, label='Bottom half (y<0)')
        plt.axhline(y=0, color='k', linestyle=':', alpha=0.5)
        plt.axis('equal')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.title('Boundary Analysis - Split Data')

        # Plot 2: Right half parabolic fits
        plt.subplot(1, 3, 2)
        plt.scatter(right_x, right_y, c='red', s=20, alpha=0.7, label='Top half data')
        if len(right_x) > 0:
            x_range = np.linspace(min(right_x), max(right_x), 50)
            y_range = np.linspace(min(right_y), max(right_y), 50)

            if right_h_r2 > right_v_r2 and right_h_coeffs is not None:
                y_fit = right_h_coeffs[0] * x_range**2 + right_h_coeffs[1] * x_range + right_h_coeffs[2]
                plt.plot(x_range, y_fit, 'blue', linewidth=2,
                        label=f'Horizontal (R²={right_h_r2:.3f})')
            elif right_v_coeffs is not None:
                x_fit = right_v_coeffs[0] * y_range**2 + right_v_coeffs[1] * y_range + right_v_coeffs[2]
                plt.plot(x_fit, y_range, 'green', linewidth=2,
                        label=f'Vertical (R²={right_v_r2:.3f})')
        plt.axis('equal')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.title('Top Half Fit (y ≥ 0)')

        # Plot 3: Left half parabolic fits
        plt.subplot(1, 3, 3)
        plt.scatter(left_x, left_y, c='orange', s=20, alpha=0.7, label='Bottom half data')
        if len(left_x) > 0:
            x_range = np.linspace(min(left_x), max(left_x), 50)
            y_range = np.linspace(min(left_y), max(left_y), 50)

            if left_h_r2 > left_v_r2 and left_h_coeffs is not None:
                y_fit = left_h_coeffs[0] * x_range**2 + left_h_coeffs[1] * x_range + left_h_coeffs[2]
                plt.plot(x_range, y_fit, 'blue', linewidth=2,
                        label=f'Horizontal (R²={left_h_r2:.3f})')
            elif left_v_coeffs is not None:
                x_fit = left_v_coeffs[0] * y_range**2 + left_v_coeffs[1] * y_range + left_v_coeffs[2]
                plt.plot(x_fit, y_range, 'green', linewidth=2,
                        label=f'Vertical (R²={left_v_r2:.3f})')
        plt.axis('equal')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.title('Bottom Half Fit (y < 0)')

        plt.tight_layout()
        plt.show()

        # Print summary
        print(f"Boundary Analysis Results:")
        print(f"- Total non-circular points: {len(parabolic_x)}")
        print(f"- Right half points (x≥0): {len(right_x)}")
        print(f"- Left half points (x<0): {len(left_x)}")
        print(f"\nRight Half Fits:")
        print(f"  - Horizontal R²: {right_h_r2:.4f}")
        print(f"  - Vertical R²: {right_v_r2:.4f}")
        print(f"\nLeft Half Fits:")
        print(f"  - Horizontal R²: {left_h_r2:.4f}")
        print(f"  - Vertical R²: {left_v_r2:.4f}")

        # Determine best fits for each half
        right_best = "horizontal" if right_h_r2 > right_v_r2 else "vertical"
        left_best = "horizontal" if left_h_r2 > left_v_r2 else "vertical"
        right_best_r2 = max(right_h_r2, right_v_r2)
        left_best_r2 = max(left_h_r2, left_v_r2)

        print(f"\nBest Fits:")
        print(f"- Right half: {right_best} parabola (R² = {right_best_r2:.4f})")
        print(f"- Left half: {left_best} parabola (R² = {left_best_r2:.4f})")

    return results

results = check_parabolic_boundaries(grid_cpu, target_values_cpu, initial_circle_radius=6, plot=True)